In [ ]:
import os, sys
#clear all variables (from last session)
sys.modules[__name__].__dict__.clear()

import os, sys, time, random, itertools
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive/ART/design+computationalthinking/code/')
from pyt_utilities import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# variables
#root = '/home/marcbohlen/'
root = '/content/drive/MyDrive/ART/design+computationalthinking/'
path = root + 'data/'
dataset = 'bali10'

#----------------------------------------
#pick a convolutional neural network model
#vanillanet, alexnet, squeezenet, resnet18, resnet152, resnext50
network = 'resnet152'
#----------------------------------------

pretrained = False;
balinorms = True;           #normalization based on the bali-26 collection. See create_norms.py

num_epochs = 40;
offset = 0; img_limit = 1000; randomprune = True;
training_percentage = 0.8;
tpp = int(100 * training_percentage)

In [ ]:
resultspath = root + 'results/'
if not os.path.exists(resultspath):
    os.makedirs(resultspath)

In [ ]:
datapath = path + dataset + '/'
categories = []

try:
    shutil.rmtree(datapath+'train')
    shutil.rmtree(datapath+'val')
except:
    pass

for _, dirs, files in os.walk(datapath):
    for dir in dirs:
        categories.append(dir)

print(categories)

In [ ]:
if(pretrained == True):
    if(balinorms == True):
        checkpointname = path + dataset + '_' + network + '_pretrainedyes_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_balinorms'+ '_checkpoint.pth'
        output = resultspath + dataset + '_' + network + '_pretrainedyes_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_balinorms'+ '.jpg'
    else:
        checkpointname = path + dataset + '_' + network + '_pretrainedyes_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_checkpoint.pth'
        output = resultspath + dataset + '_' + network + '_pretrainedyes_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '.jpg'
else:
    if(balinorms == True):
        checkpointname = path + dataset + '_' + network + '_pretrainedNO_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_balinorms'+ '_checkpoint.pth'
        output = resultspath + dataset + '_' + network + '_pretrainedyNO_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_balinorms'+ '.jpg'
    else:
        checkpointname = path + dataset + '_' + network + '_pretrainedNO_t' + str(tpp) + '_e' + str(num_epochs) +'_lim' + str(img_limit) +  '_checkpoint.pth'
        output = resultspath + dataset + '_' + network + '_pretrainedNO_t' + str(tpp) + '_e' + str(num_epochs) +'_lim' + str(img_limit) +  '.jpg'


In [ ]:
create_train_val_sets(datapath, categories, training_percentage)

In [ ]:
image_datasets = {x: datasets.ImageFolder(os.path.join(datapath, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = ("cuda" if torch.cuda.is_available() else "cpu" )

In [ ]:
if(network == 'vanillanet'):
    model = vanillanet(len(class_names))

elif(network == 'alexnet'):
    model = models.alexnet(pretrained=pretrained)
    model.classifier[6] = torch.nn.Linear(4096,len(class_names))

elif(network == 'squeezenet'):
    model = models.squeezenet1_1(pretrained=pretrained)
    model.classifier[1] = torch.nn.Conv2d(512, len(class_names), kernel_size=(1,1), stride=(1,1))

elif(network == 'resnet18'):
    model = models.resnet18(pretrained=pretrained)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, len(class_names))

elif(network == 'resnet152'):
    model = models.resnet152(pretrained=pretrained)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, len(class_names))

elif(network == 'resnext50'):
    model = models.resnext50_32x4d(pretrained=pretrained)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, len(class_names))
else:
    print('something went wrong setting up the selected model..')
    exit()

In [ ]:
print('\nthe dataset is: ', dataset)
print('the network is: ', network)
print('pretrained network: ', pretrained)
print('training and validation distribution: ', tpp, (100-tpp))
print('number of training epochs set to: ', num_epochs)
print('the engine is: ', device)

In [ ]:
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model = train_model(checkpointname, model, dataloaders, dataset_sizes, criterion, optimizer_ft, exp_lr_scheduler, device, num_epochs, output)
print('Finished Training')